In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import json
import ccxt

In [2]:
def convert_tz(utc):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    utc = utc.replace(tzinfo=from_zone).astimezone(to_zone)
    
    return utc

In [3]:
def get_date_list(start_date, end_date=None):
    '''
    Generate list of date to fetch 1 day iteration from fetch_ohlcv.
    '''
    if end_date == None:
        end_date = dt.date.today()
    
    num_day = (end_date - start_date).days
    date_list = [dt.date.today() - dt.timedelta(days = x) for x in range(num_day, 0, -1)]
    
    return date_list

In [4]:
def get_js_date(dt_date):
    '''
    Transform dt.datetime to JavaScript format.
    '''
    dt_datetime = dt.datetime(dt_date.year, dt_date.month, dt_date.day)
    js_datetime = dt_datetime.timestamp() * 1000
    
    return js_datetime

In [5]:
def get_ohlcv(symbol, date_list, timeframe, limit):
    df = pd.DataFrame(columns = ['time', 'open', 'high', 'low', 'close', 'volume'])

    for i in range(len(date_list)):
        since = get_ts_date(date_list[i])
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        temp_df = pd.DataFrame(ohlcv)
        temp_df.columns = [['time', 'open', 'high', 'low', 'close', 'volume']]
        temp_df['time'] = pd.to_datetime(temp_df['time'], unit='ms')
        temp_df['time'] = temp_df['time'].apply(lambda x: convert_tz(x))

        df = pd.concat([df, temp_df])

    df = df.reset_index(drop = True)

    return df

### Config 

In [6]:
with open('../../../_keys/ftx/ftx_main_keys.json') as keys_file:
    keys_dict = json.load(keys_file)

In [7]:
exchange = ccxt.ftx({'apiKey': keys_dict['apiKey'],
                     'secret': keys_dict['secret'],
                     'enableRateLimit': True})

In [8]:
file_path = 'data'

In [10]:
timeframe = '1m'
limit = 60 * 24

### Get data

In [27]:
base_currency = 'BTC'
quote_currency = 'USDT'

In [28]:
start_date = dt.date(2021, 1, 1)
date_list = get_date_list(start_date)

In [29]:
symbol = '{}/{}'.format(base_currency, quote_currency)
df = get_ohlcv(symbol, date_list, timeframe, limit)

In [30]:
df.to_csv('{}/{}-{}.csv'.format(file_path, base_currency, quote_currency), index=False)